In [1]:
import pandas as pd
import numpy as np
from scipy import stats as stats
import regex
import pickle
from matplotlib import pyplot as plt
import requests
from bs4 import BeautifulSoup as bs
import datetime

In [2]:
years=['2019','2018','2017','2016','2015']
data={}

In [3]:
for yr in years:
    data[yr]=pd.read_excel('../Raw Data/'+yr+'.xlsx')

In [4]:
for yr,df in data.items():
    df_clean=df.drop(['Rot','1st','2nd','3rd','4th','5th','6th','7th','8th','9th','Open','Close','RunLine','Run Line','Unnamed: 18'],axis=1,errors='ignore')
    df_clean['year']=int(yr)
    df_clean['day']=df_clean.Date.apply(lambda x:x%100)
    df_clean['month']=df_clean.Date.apply(lambda x:int(np.round(x,-2)/100))
    df_clean['date']=df_clean.apply(lambda r:datetime.datetime(r.year,r.month,r.day),axis=1)
    df_clean.drop(['Date','day'],axis=1,inplace=True)
    data[yr]=df_clean.rename({'Open OU':'OpenOU','Close OU':'CloseOU','Unnamed: 20':'Open_Odds','Unnamed: 22':'Close_Odds'},axis=1)

In [5]:
all_data=pd.concat(data.values(),ignore_index=True)

In [6]:
patt=r'\-[A-Z]{1}'
all_data.Pitcher=all_data.Pitcher.str.replace(patt,'')

In [7]:
all_data.Team.value_counts()

CUB    811
MIL    811
COL    811
BAL    810
WAS    810
SEA    810
NYM    810
TEX    810
CIN    810
OAK    810
PHI    810
ARI    810
LAA    810
TOR    810
SFO    810
HOU    810
PIT    810
MIN    810
SDG    809
ATL    809
TAM    809
BOS    809
CWS    809
NYY    809
KAN    809
CLE    808
STL    808
MIA    808
DET    807
LOS    485
LAD    326
Name: Team, dtype: int64

In [8]:
all_data.Team.replace({'CUB':'CHC','LOS':'LAD','SFG':'SFO','BRS':'BOS'}).value_counts()

LAD    811
CHC    811
MIL    811
COL    811
LAA    810
WAS    810
SEA    810
NYM    810
TEX    810
CIN    810
OAK    810
PHI    810
BAL    810
ARI    810
HOU    810
PIT    810
MIN    810
SFO    810
TOR    810
CWS    809
ATL    809
NYY    809
KAN    809
SDG    809
TAM    809
BOS    809
MIA    808
STL    808
CLE    808
DET    807
Name: Team, dtype: int64

In [9]:
all_data.Team=all_data.Team.replace({'CUB':'CHC','LOS':'LAD','SFG':'SFO','BRS':'BOS'})

In [10]:
all_data.Team=all_data.Team.replace({
                                    'KAN':'KCR',
                                    'TAM':'TBR',
                                    'SDG':'SDP',
                                    'WAS':'WSN',
                                    'SFO':'SFG',
                                    'CWS':'CHW'
                                    })

In [11]:
all_data

,VH,Team,Pitcher,Final,OpenOU,Open_Odds,CloseOU,Close_Odds,year,month,date
0,N,SEA,MGONZALES,9,8.5,-110,8.5,-120,2019,3,2019-03-20
1,N,OAK,MFIERS,7,8.5,-110,8.5,100,2019,3,2019-03-20
2,N,SEA,YKIKUCHI,5,9.0,-110,9.0,-130,2019,3,2019-03-21
3,N,OAK,MESTRADA,4,9.0,-110,9.0,110,2019,3,2019-03-21
4,V,NYM,JDEGROM,2,6.5,-110,6.5,-105,2019,3,2019-03-28
...,...,...,...,...,...,...,...,...,...,...,...
24283,H,CHW,FMONTAS,0,8.5,-110,7.5,-130,2015,10,2015-10-04
24284,V,HOU,MCCULLERS,3,8.5,-105,8.5,-105,2015,10,2015-10-04
24285,H,ARI,RRAY,5,8.5,-115,8.5,-115,2015,10,2015-10-04
24286,V,STL,JLACKEY,0,7.0,105,7.0,105,2015,10,2015-10-04


In [12]:
all_data.VH.value_counts()

V    12142
H    12142
N        4
Name: VH, dtype: int64

In [13]:
all_data[all_data.VH=='N']

,VH,Team,Pitcher,Final,OpenOU,Open_Odds,CloseOU,Close_Odds,year,month,date
0,N,SEA,MGONZALES,9,8.5,-110,8.5,-120,2019,3,2019-03-20
1,N,OAK,MFIERS,7,8.5,-110,8.5,100,2019,3,2019-03-20
2,N,SEA,YKIKUCHI,5,9.0,-110,9.0,-130,2019,3,2019-03-21
3,N,OAK,MESTRADA,4,9.0,-110,9.0,110,2019,3,2019-03-21


In [14]:
error_01='https://www.baseball-reference.com/boxes/OAK/OAK201903200.shtml'
error_23='https://www.baseball-reference.com/boxes/OAK/OAK201903210.shtml'

In [15]:
all_data.at[0,'VH']='V'
all_data.at[1,'VH']='H'
all_data.at[2,'VH']='V'
all_data.at[3,'VH']='H'

In [16]:
all_data.head()

,VH,Team,Pitcher,Final,OpenOU,Open_Odds,CloseOU,Close_Odds,year,month,date
0,V,SEA,MGONZALES,9,8.5,-110,8.5,-120,2019,3,2019-03-20
1,H,OAK,MFIERS,7,8.5,-110,8.5,100,2019,3,2019-03-20
2,V,SEA,YKIKUCHI,5,9.0,-110,9.0,-130,2019,3,2019-03-21
3,H,OAK,MESTRADA,4,9.0,-110,9.0,110,2019,3,2019-03-21
4,V,NYM,JDEGROM,2,6.5,-110,6.5,-105,2019,3,2019-03-28


In [17]:
all_data.VH.value_counts()

V    12144
H    12144
Name: VH, dtype: int64

In [18]:
all_data.Final.value_counts()

3     3313
2     3207
4     3137
5     2779
1     2571
6     2151
7     1677
0     1457
8     1267
9      831
10     650
11     444
12     292
13     185
14     124
15      73
16      48
17      34
19      16
18      13
21       7
20       4
22       2
23       2
NL       2
25       1
24       1
Name: Final, dtype: int64

In [19]:
all_data[all_data.Final=='NL']

,VH,Team,Pitcher,Final,OpenOU,Open_Odds,CloseOU,Close_Odds,year,month,date
16276,V,NYM,SMATZ,NL,8.0,100,7.5,-110,2016,6,2016-06-06
16277,H,PIT,JNIESE,NL,8.0,-120,7.5,-110,2016,6,2016-06-06


In [20]:
error_Final='https://www.baseball-reference.com/boxes/PIT/PIT201606071.shtml'
all_data.at[16276,'Final']=3
all_data.at[16277,'Final']=3

In [21]:
all_data.Final.value_counts()

3     3315
2     3207
4     3137
5     2779
1     2571
6     2151
7     1677
0     1457
8     1267
9      831
10     650
11     444
12     292
13     185
14     124
15      73
16      48
17      34
19      16
18      13
21       7
20       4
22       2
23       2
24       1
25       1
Name: Final, dtype: int64

In [22]:
all_data.Final=all_data.Final.astype(int)

In [23]:
all_data.CloseOU.value_counts()

8.5      4710
9.0      4402
8.0      3596
7.5      3334
9.5      2466
7.0      2004
10.0     1172
10.5      902
6.5       550
11.0      470
11.5      226
6.0       138
12.0      108
12.5       76
13.0       56
13.5       30
5.5        16
14.0       14
14.5        6
102.0       2
15.0        2
104.0       2
5.0         2
101.0       2
100.0       2
Name: CloseOU, dtype: int64

In [24]:
all_data[all_data.CloseOU > 20]

,VH,Team,Pitcher,Final,OpenOU,Open_Odds,CloseOU,Close_Odds,year,month,date
5872,V,DET,MFULMER,6,9.5,-110,104.0,-110,2018,5,2018-05-07
5873,H,TEX,MMOORE,7,9.5,-110,104.0,-110,2018,5,2018-05-07
6740,V,STL,MWACHA,6,8.5,-120,102.0,-110,2018,6,2018-06-09
6741,H,CIN,LCASTILLO,4,8.5,100,102.0,-110,2018,6,2018-06-09
8842,V,SEA,ERAMIREZ,3,8.0,-110,101.0,-110,2018,8,2018-08-29
8843,H,SDP,JLUCCHESI,8,8.0,-110,101.0,-110,2018,8,2018-08-29
9478,V,KCR,JJUNIS,4,8.5,-110,100.0,-110,2018,9,2018-09-22
9479,H,DET,JZIMMERMANN,5,8.5,-110,100.0,-110,2018,9,2018-09-22


In [25]:
all_data.CloseOU=all_data.apply(lambda r:r.OpenOU if r.CloseOU>20 else r.CloseOU,axis=1)

In [26]:
all_data.CloseOU.value_counts()

8.5     4714
9.0     4402
8.0     3598
7.5     3334
9.5     2468
7.0     2004
10.0    1172
10.5     902
6.5      550
11.0     470
11.5     226
6.0      138
12.0     108
12.5      76
13.0      56
13.5      30
5.5       16
14.0      14
14.5       6
5.0        2
15.0       2
Name: CloseOU, dtype: int64

In [27]:
all_data.CloseOU=all_data.CloseOU.astype(float)

In [28]:
all_data['Home_Pitcher']=''
all_data['Away_Pitcher']=''
all_data['Home_Team']=''
all_data['Away_Team']=''
all_data['Home_Score']=0
all_data['Away_Score']=0
for i,row in all_data.iterrows():
    if row.VH=='V':
        all_data.loc[i,'Away_Pitcher']=all_data.loc[i,'Pitcher']
        all_data.loc[i,'Away_Team']=all_data.loc[i,'Team']
        all_data.loc[i,'Away_Score']=all_data.loc[i,'Final']
        all_data.loc[i,'Home_Score']=all_data.loc[i+1,'Final']
        all_data.loc[i,'Home_Team']=all_data.loc[i+1,'Team']
        all_data.loc[i,'Home_Pitcher']=all_data.loc[i+1,'Pitcher']
        pass
    elif row.VH=='H':
        all_data.loc[i,'Away_Pitcher']=all_data.loc[i-1,'Pitcher']
        all_data.loc[i,'Away_Team']=all_data.loc[i-1,'Team']
        all_data.loc[i,'Away_Score']=all_data.loc[i-1,'Final']
        all_data.loc[i,'Home_Score']=all_data.loc[i,'Final']
        all_data.loc[i,'Home_Team']=all_data.loc[i,'Team']
        all_data.loc[i,'Home_Pitcher']=all_data.loc[i,'Pitcher']
        pass
    else:
        pass
all_data

,VH,Team,Pitcher,Final,OpenOU,Open_Odds,CloseOU,Close_Odds,year,month,date,Home_Pitcher,Away_Pitcher,Home_Team,Away_Team,Home_Score,Away_Score
0,V,SEA,MGONZALES,9,8.5,-110,8.5,-120,2019,3,2019-03-20,MFIERS,MGONZALES,OAK,SEA,7,9
1,H,OAK,MFIERS,7,8.5,-110,8.5,100,2019,3,2019-03-20,MFIERS,MGONZALES,OAK,SEA,7,9
2,V,SEA,YKIKUCHI,5,9.0,-110,9.0,-130,2019,3,2019-03-21,MESTRADA,YKIKUCHI,OAK,SEA,4,5
3,H,OAK,MESTRADA,4,9.0,-110,9.0,110,2019,3,2019-03-21,MESTRADA,YKIKUCHI,OAK,SEA,4,5
4,V,NYM,JDEGROM,2,6.5,-110,6.5,-105,2019,3,2019-03-28,MSCHERZER,JDEGROM,WSN,NYM,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24283,H,CHW,FMONTAS,0,8.5,-110,7.5,-130,2015,10,2015-10-04,FMONTAS,DNORRIS,CHW,DET,0,6
24284,V,HOU,MCCULLERS,3,8.5,-105,8.5,-105,2015,10,2015-10-04,RRAY,MCCULLERS,ARI,HOU,5,3
24285,H,ARI,RRAY,5,8.5,-115,8.5,-115,2015,10,2015-10-04,RRAY,MCCULLERS,ARI,HOU,5,3
24286,V,STL,JLACKEY,0,7.0,105,7.0,105,2015,10,2015-10-04,SMILLER,JLACKEY,ATL,STL,6,0


In [34]:
all_data=all_data.drop_duplicates(subset=['Away_Pitcher','Home_Pitcher','date'],ignore_index=True)
all_data

,VH,Team,Pitcher,Final,OpenOU,Open_Odds,CloseOU,Close_Odds,year,month,date,Home_Pitcher,Away_Pitcher,Home_Team,Away_Team,Home_Score,Away_Score
0,V,SEA,MGONZALES,9,8.5,-110,8.5,-120,2019,3,2019-03-20,MFIERS,MGONZALES,OAK,SEA,7,9
1,V,SEA,YKIKUCHI,5,9.0,-110,9.0,-130,2019,3,2019-03-21,MESTRADA,YKIKUCHI,OAK,SEA,4,5
2,V,NYM,JDEGROM,2,6.5,-110,6.5,-105,2019,3,2019-03-28,MSCHERZER,JDEGROM,WSN,NYM,0,2
3,V,STL,MMIKOLAS,4,8.5,-110,8.5,-120,2019,3,2019-03-28,JCHACIN,MMIKOLAS,MIL,STL,5,4
4,V,ATL,JTEHERAN,4,7.5,-110,8.0,-105,2019,3,2019-03-28,ANOLA,JTEHERAN,PHI,ATL,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,V,OAK,CBASSITT,2,8.0,-105,7.5,-115,2015,10,2015-10-04,VNUNO,CBASSITT,SEA,OAK,3,2
12140,V,KCR,JCUETO,6,8.0,-105,7.5,-120,2015,10,2015-10-04,RNOLASCO,JCUETO,MIN,KCR,1,6
12141,V,DET,DNORRIS,6,8.5,-110,7.5,110,2015,10,2015-10-04,FMONTAS,DNORRIS,CHW,DET,0,6
12142,V,HOU,MCCULLERS,3,8.5,-105,8.5,-105,2015,10,2015-10-04,RRAY,MCCULLERS,ARI,HOU,5,3


In [35]:
all_data.drop(['VH','Team','Pitcher','Final','OpenOU','Open_Odds'],axis=1,inplace=True)
all_data

,CloseOU,Close_Odds,year,month,date,Home_Pitcher,Away_Pitcher,Home_Team,Away_Team,Home_Score,Away_Score
0,8.5,-120,2019,3,2019-03-20,MFIERS,MGONZALES,OAK,SEA,7,9
1,9.0,-130,2019,3,2019-03-21,MESTRADA,YKIKUCHI,OAK,SEA,4,5
2,6.5,-105,2019,3,2019-03-28,MSCHERZER,JDEGROM,WSN,NYM,0,2
3,8.5,-120,2019,3,2019-03-28,JCHACIN,MMIKOLAS,MIL,STL,5,4
4,8.0,-105,2019,3,2019-03-28,ANOLA,JTEHERAN,PHI,ATL,10,4
...,...,...,...,...,...,...,...,...,...,...,...
12139,7.5,-115,2015,10,2015-10-04,VNUNO,CBASSITT,SEA,OAK,3,2
12140,7.5,-120,2015,10,2015-10-04,RNOLASCO,JCUETO,MIN,KCR,1,6
12141,7.5,110,2015,10,2015-10-04,FMONTAS,DNORRIS,CHW,DET,0,6
12142,8.5,-105,2015,10,2015-10-04,RRAY,MCCULLERS,ARI,HOU,5,3


In [36]:
all_data['Total_Runs']=all_data['Home_Score']+all_data['Away_Score']
all_data['OVER']=all_data.apply(lambda r:r.Total_Runs>r.CloseOU,axis=1)
all_data

,CloseOU,Close_Odds,year,month,date,Home_Pitcher,Away_Pitcher,Home_Team,Away_Team,Home_Score,Away_Score,Total_Runs,OVER
0,8.5,-120,2019,3,2019-03-20,MFIERS,MGONZALES,OAK,SEA,7,9,16,True
1,9.0,-130,2019,3,2019-03-21,MESTRADA,YKIKUCHI,OAK,SEA,4,5,9,False
2,6.5,-105,2019,3,2019-03-28,MSCHERZER,JDEGROM,WSN,NYM,0,2,2,False
3,8.5,-120,2019,3,2019-03-28,JCHACIN,MMIKOLAS,MIL,STL,5,4,9,True
4,8.0,-105,2019,3,2019-03-28,ANOLA,JTEHERAN,PHI,ATL,10,4,14,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,7.5,-115,2015,10,2015-10-04,VNUNO,CBASSITT,SEA,OAK,3,2,5,False
12140,7.5,-120,2015,10,2015-10-04,RNOLASCO,JCUETO,MIN,KCR,1,6,7,False
12141,7.5,110,2015,10,2015-10-04,FMONTAS,DNORRIS,CHW,DET,0,6,6,False
12142,8.5,-105,2015,10,2015-10-04,RRAY,MCCULLERS,ARI,HOU,5,3,8,False


In [44]:
all_data['KEY']=all_data.date.apply(lambda d:str(d).replace(' 00:00:00','-'))+all_data.Home_Team

In [45]:
all_data[all_data.KEY.duplicated(keep=False)]

,CloseOU,Close_Odds,year,month,date,Home_Pitcher,Away_Pitcher,Home_Team,Away_Team,Home_Score,Away_Score,Total_Runs,OVER,KEY
303,10.0,-105,2019,4,2019-04-20,ACOBB,MPEREZ,BAL,MIN,7,16,23,True,2019-04-20-BAL
306,8.5,100,2019,4,2019-04-20,CKLUBER,JTEHERAN,CLE,ATL,8,4,12,True,2019-04-20-CLE
307,8.0,-115,2019,4,2019-04-20,TBAUER,UNDECIDED,CLE,ATL,7,8,15,True,2019-04-20-CLE
308,9.0,-120,2019,4,2019-04-20,DSTRAILY,JBERRIOS,BAL,MIN,5,6,11,True,2019-04-20-BAL
341,9.0,-115,2019,4,2019-04-23,HVELAZQUEZ,STURNBULL,BOS,DET,2,4,6,False,2019-04-23-BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12125,7.5,105,2015,10,2015-10-03,AHARANG,TKOEHLER,PHI,MIA,6,7,13,True,2015-10-03-PHI
12126,7.5,-110,2015,10,2015-10-03,WCHEN,INOVA,BAL,NYY,9,2,11,True,2015-10-03-BAL
12127,7.0,107,2015,10,2015-10-03,SYNDERGARD,GGONZALEZ,NYM,WSN,1,3,4,False,2015-10-03-NYM
12128,7.5,100,2015,10,2015-10-04,MWISLER,LLYNN,ATL,STL,2,0,2,False,2015-10-04-ATL


In [46]:
all_data.KEY=all_data.apply(lambda r:r.KEY+str(r.Home_Score)+str(r.Away_Score),axis=1)
all_data[all_data.KEY.duplicated(keep=False)]

,CloseOU,Close_Odds,year,month,date,Home_Pitcher,Away_Pitcher,Home_Team,Away_Team,Home_Score,Away_Score,Total_Runs,OVER,KEY
8147,7.5,-120,2016,6,2016-06-07,JNIESE,SMATZ,PIT,NYM,3,1,4,False,2016-06-07-PIT31
8162,7.5,100,2016,6,2016-06-07,JNICASIO,JDEGROM,PIT,NYM,3,1,4,False,2016-06-07-PIT31


In [47]:
all_data.at[8147,'KEY']=all_data.at[8147,'KEY']='A'
all_data.at[8162,'KEY']=all_data.at[8162,'KEY']='B'

In [48]:
all_data

,CloseOU,Close_Odds,year,month,date,Home_Pitcher,Away_Pitcher,Home_Team,Away_Team,Home_Score,Away_Score,Total_Runs,OVER,KEY
0,8.5,-120,2019,3,2019-03-20,MFIERS,MGONZALES,OAK,SEA,7,9,16,True,2019-03-20-OAK79
1,9.0,-130,2019,3,2019-03-21,MESTRADA,YKIKUCHI,OAK,SEA,4,5,9,False,2019-03-21-OAK45
2,6.5,-105,2019,3,2019-03-28,MSCHERZER,JDEGROM,WSN,NYM,0,2,2,False,2019-03-28-WSN02
3,8.5,-120,2019,3,2019-03-28,JCHACIN,MMIKOLAS,MIL,STL,5,4,9,True,2019-03-28-MIL54
4,8.0,-105,2019,3,2019-03-28,ANOLA,JTEHERAN,PHI,ATL,10,4,14,True,2019-03-28-PHI104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,7.5,-115,2015,10,2015-10-04,VNUNO,CBASSITT,SEA,OAK,3,2,5,False,2015-10-04-SEA32
12140,7.5,-120,2015,10,2015-10-04,RNOLASCO,JCUETO,MIN,KCR,1,6,7,False,2015-10-04-MIN16
12141,7.5,110,2015,10,2015-10-04,FMONTAS,DNORRIS,CHW,DET,0,6,6,False,2015-10-04-CHW06
12142,8.5,-105,2015,10,2015-10-04,RRAY,MCCULLERS,ARI,HOU,5,3,8,False,2015-10-04-ARI53


In [52]:
all_data.to_pickle('../Base DFs/Odds_Data.pkl')